In [1]:
import cv2
import mediapipe as mp
import time
import math
from enum import Enum
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter, find_peaks
from collections import deque

# Define Enum for specific keypoints (shoulders and hips)
class Keypoints(Enum):
    LEFT_SHOULDER = 11
    RIGHT_SHOULDER = 12
    LEFT_WRIST = 15
    RIGHT_WRIST = 16

class poseDetector():
    def __init__(self, mode=False, upBody=True, smooth=True,
                 detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.upBody = upBody
        self.smooth = smooth
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        self.mpDraw = mp.solutions.drawing_utils
        self.mpPose = mp.solutions.pose
        self.lmList = []
        self.right_arm_angles = deque(maxlen=5)  # Stores recent right elbow angles
        self.left_arm_angles = deque(maxlen=5)  # Stores recent left elbow angles
        self.wrist_distances = deque(maxlen=5)  # Store recent wrist distances
        
        # Pose constructor
        self.pose = self.mpPose.Pose(
            static_image_mode=self.mode,
            model_complexity=1,
            smooth_landmarks=self.smooth,
            enable_segmentation=False,
            min_detection_confidence=self.detectionCon,
            min_tracking_confidence=self.trackCon
        )
        
        # List to store midpoints between hips
        self.midpoints = []
        self.midpoints_y_values = []
        
    def findPose(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.pose.process(imgRGB)
        
        return img

    def findPosition(self, img, draw=True):
        lmList = []
        if self.results.pose_landmarks:
            for id, lm in enumerate(self.results.pose_landmarks.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                visibility = lm.visibility
                lmList.append([id, cx, cy, visibility])
                self.lmList = lmList

        # Only return and print the keypoints of interest (shoulders and hips)
        if draw:
            keypoints = [Keypoints.LEFT_SHOULDER, Keypoints.RIGHT_SHOULDER, Keypoints.LEFT_WRIST, Keypoints.RIGHT_WRIST]
            points = {}

            for keypoint in keypoints:
                id = keypoint.value
                cx, cy = lmList[id][1], lmList[id][2]
                visibility = lmList[id][3]
                points[keypoint] = (cx, cy, visibility)
                
                # Draw the keypoints
                cv2.circle(img, (cx, cy), 10, (0, 255, 0), cv2.FILLED)
                cv2.putText(img, f'{keypoint.name}', (cx + 10, cy + 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)
                
                # Print x, y, and visibility for the keypoints
                print(f'{keypoint.name} - X: {cx}, Y: {cy}, Visibility: {visibility}')

            # Draw connections between the keypoints (shoulders and hips)
            cv2.line(img, points[Keypoints.LEFT_SHOULDER][:2], points[Keypoints.LEFT_WRIST][:2], (255, 0, 0), 2)
            cv2.line(img, points[Keypoints.RIGHT_SHOULDER][:2], points[Keypoints.RIGHT_WRIST][:2], (255, 0, 0), 2)
            cv2.line(img, points[Keypoints.LEFT_SHOULDER][:2], points[Keypoints.RIGHT_SHOULDER][:2], (255, 0, 0), 2)
            cv2.line(img, points[Keypoints.LEFT_WRIST][:2], points[Keypoints.RIGHT_WRIST][:2], (255, 0, 0), 2)

            # Calculate the midpoint between the two hips
            left_wrist = points[Keypoints.LEFT_WRIST][:2]
            right_wrist = points[Keypoints.RIGHT_WRIST][:2]
            midpoint = ((left_wrist[0] + right_wrist[0]) // 2, (left_wrist[1] + right_wrist[1]) // 2)
            midpoints_y_value = (left_wrist[1] + right_wrist[1]) // 2
            
            # Append the midpoint to the midpoints list
            self.midpoints.append(midpoint)
            self.midpoints_y_values.append(midpoints_y_value)


            
            # Draw the midpoint
            cv2.circle(img, midpoint, 10, (255, 255, 0), cv2.FILLED)
            cv2.putText(img, 'Midpoint', (midpoint[0] + 10, midpoint[1] + 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 0), 2)

        return lmList, points

    def findAngle(self, img, p1, p2, p3, draw=True):
        x1, y1 = self.lmList[p1][1:3]
        x2, y2 = self.lmList[p2][1:3]
        x3, y3 = self.lmList[p3][1:3]
        
        angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - 
                             math.atan2(y1 - y2, x1 - x2))
        if angle < 0:
            angle += 360
        
        if draw:
            cv2.line(img, (x1, y1), (x2, y2), (255, 255, 255), 3)
            cv2.line(img, (x3, y3), (x2, y2), (255, 255, 255), 3)
            cv2.circle(img, (x1, y1), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x1, y1), 15, (0, 0, 255), 2)
            cv2.circle(img, (x2, y2), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), 15, (0, 0, 255), 2)
            cv2.circle(img, (x3, y3), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x3, y3), 15, (0, 0, 255), 2)
            cv2.putText(img, str(int(angle)), (x2 - 50, y2 + 50),
                        cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
        return angle
    
    #Function that uses the list of midpoints to draw a curve over a certain number of frames (list size) using matplotlib
    def drawCurve(self):
        import matplotlib.pyplot as plt

        # Extract y coordinates from midpoints
        y_coords = [point[1] for point in self.midpoints]
        x_coords = list(range(len(y_coords)))  # Use the index as x coordinates

        # Plot the curve
        plt.figure(figsize=(10, 5))
        plt.plot(x_coords, y_coords, marker='o', color='b')
        plt.title('Midpoint Curve')
        plt.xlabel('Frame Index')
        plt.ylabel('Y Coordinates')
        plt.grid(True)
        plt.show()

    # Draw the curve of midpoints and draw on the same plot the reference curve
    def drawCurveWithReference(self, peaks, smoothed_y):
        # Extract y coordinates from midpoints
        y_coords = [point[1] for point in self.midpoints]
        x_coords = list(range(len(y_coords)))  # Use the index as x coordinates

        # Plot the curve
        plt.figure(figsize=(10, 5))
        plt.plot(x_coords, y_coords, color='red', label='Midpoint Curve')
        plt.plot(smoothed_y, label="Smoothed Motion", color="blue", linewidth=2)
        plt.plot(peaks, smoothed_y[peaks], "x", color="green", label='Reference Curve')
        plt.title('Midpoint Curve')
        plt.xlabel('Frame Index')
        plt.ylabel('Y Coordinates')
        plt.grid(True)
        plt.legend()
        plt.show()

    def is_right_arm_bent(self, img, elbow, shoulder, wrist):
        """
        Checks if the arm is bent based on the elbow angle.
      
        """
        angle = self.findAngle(img, wrist, elbow, shoulder)
        self.right_arm_angles.append(angle)  # Store latest angle
        avg_angle = np.mean(self.right_arm_angles)  # Compute smoothed angle

        return avg_angle > 210  # Consider bent if above this threshold
    

    def is_left_arm_bent(self, img, elbow, shoulder, wrist):
        """
        Checks if the arm is bent based on the elbow angle.
    
        """
        angle = self.findAngle(img, wrist, elbow, shoulder)
        self.left_arm_angles.append(angle)  # Store latest angle
        avg_angle = np.mean(self.left_arm_angles)  # Compute smoothed angle

        return avg_angle < 150  # Consider bent if below this threshold



    def check_arms_bent(self, img):
        """
        Determines if one or both arms are bent using elbow angles.
        """
        left_bent = self.is_left_arm_bent(img, 13, 11, 15)  # Left arm: (elbow, shoulder, wrist)


        right_bent = self.is_right_arm_bent(img, 14, 12, 16)  # Right arm: (elbow, shoulder, wrist)

        if left_bent and right_bent:
            return "Both arms are bent"
        elif left_bent:
            return "Left arm is bent"
        elif right_bent:
            return "Right arm is bent"
        else:
            return None

    def is_one_handed_cpr(self, points, frame_width):
        """
        Checks if the person is using only one hand.
        """
        if Keypoints.LEFT_WRIST not in points or Keypoints.RIGHT_WRIST not in points:
            return False  # If we don't detect both wrists, assume not one-handed

        # Calculate the distance between wrists
        wrist_distance = abs(points[Keypoints.LEFT_WRIST][0] - points[Keypoints.RIGHT_WRIST][0])
        
        # Store the wrist distance in the queue
        self.wrist_distances.append(wrist_distance)

        # Compute smoothed wrist distance
        avg_wrist_distance = np.mean(self.wrist_distances)

        # Return True if the average wrist distance is above threshold
        return avg_wrist_distance > 170  #! will need to adjust this threshold based on the video PX TO CM CONVERSION

    def process_frame(self, img):
        self.findPose(img)
        lmList, points = self.findPosition(img)

        if points:
            h, w, _ = img.shape

            # Check arms bent
            arm_status = self.check_arms_bent(img)
            if arm_status:
                cv2.putText(img, f"Warning: {arm_status}", (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

            # Check one-handed CPR
            if self.is_one_handed_cpr(points, w):
                cv2.putText(img, "Warning: One-Handed CPR detected!", (50, 100),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        return img


    def track_motion(self, img, points):
        """
        Tracks CPR motion by storing midpoint positions.
        """
        left_wrist, right_wrist = points[Keypoints.LEFT_WRIST], points[Keypoints.RIGHT_WRIST]
        midpoint = ((left_wrist[0] + right_wrist[0]) // 2, (left_wrist[1] + right_wrist[1]) // 2)

        self.midpoints.append(midpoint)
        self.midpoints_y_values.append(midpoint[1])

        # Draw midpoint
        cv2.circle(img, midpoint, 10, (255, 255, 0), cv2.FILLED)
        cv2.putText(img, "Midpoint", (midpoint[0] + 10, midpoint[1] + 10),
                    cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 0), 2)

    def plot_motion(self):
        """
        Plots CPR motion over time.
        """
        x_coords = list(range(len(self.midpoints_y_values)))
        smoothed_y = savgol_filter(self.midpoints_y_values, 7, 3)  # Smooth the motion

        plt.figure(figsize=(10, 5))
        plt.plot(x_coords, smoothed_y, label="Smoothed Motion", color="blue", linewidth=2)
        peaks, _ = find_peaks(smoothed_y, distance=5)
        plt.plot(peaks, smoothed_y[peaks], "x", color="red", label="Peaks")
        
        plt.title("CPR Motion Over Time")
        plt.xlabel("Frame Index")
        plt.ylabel("Midpoint Y-Position")
        plt.legend()
        plt.grid(True)
        plt.show()
   


In [2]:
cap = cv2.VideoCapture("Posture.mp4")  # Use 0 for webcam
detector = poseDetector()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame. Exiting loop.")
        break  # Exit loop safely

    try:
        frame = detector.process_frame(frame)
        if frame is None:
            continue  # Skip frame if processing failed

        cv2.imshow("CPR Posture Tracking", frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            print("User exited. Closing video.")
            break  # Exit if user presses 'q'

    except IndexError as e:
        print(f"Error: {e}. Skipping frame.")
        continue  # Skip this frame safely

cap.release()
cv2.destroyAllWindows()

# Ensure there's valid motion data before plotting
if hasattr(detector, "motion_data") and detector.motion_data:
    detector.plot_motion()
else:
    print("No motion data recorded. Skipping plot.")


LEFT_SHOULDER - X: 374, Y: 247, Visibility: 0.9999610185623169
RIGHT_SHOULDER - X: 162, Y: 340, Visibility: 0.9999396800994873
LEFT_WRIST - X: 495, Y: 360, Visibility: 0.9021893739700317
RIGHT_WRIST - X: 248, Y: 613, Visibility: 0.86602783203125
LEFT_SHOULDER - X: 374, Y: 251, Visibility: 0.9999573230743408
RIGHT_SHOULDER - X: 161, Y: 341, Visibility: 0.9999328851699829
LEFT_WRIST - X: 495, Y: 367, Visibility: 0.9012230038642883
RIGHT_WRIST - X: 243, Y: 612, Visibility: 0.8660362958908081
LEFT_SHOULDER - X: 376, Y: 263, Visibility: 0.999894917011261
RIGHT_SHOULDER - X: 162, Y: 346, Visibility: 0.9999178051948547
LEFT_WRIST - X: 484, Y: 389, Visibility: 0.8450764417648315
RIGHT_WRIST - X: 233, Y: 608, Visibility: 0.8490473628044128
LEFT_SHOULDER - X: 377, Y: 278, Visibility: 0.9998868703842163
RIGHT_SHOULDER - X: 165, Y: 355, Visibility: 0.9999122619628906
LEFT_WRIST - X: 471, Y: 419, Visibility: 0.8475034832954407
RIGHT_WRIST - X: 225, Y: 604, Visibility: 0.8528364300727844
LEFT_SHOULD